In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('../Grunddatein/Zwischendatein/CleanedDataCompleteNoNulls.csv')
df.info()

C:\Users\Robert Betschinger\AppData\Local\Temp\ipykernel_1600\1313368320.py:4: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../Grunddatein/Zwischendatein/CleanedDataCompleteNoNulls.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6591111 entries, 0 to 6591110
Data columns (total 20 columns):
 #   Column            Dtype  
---  ------            -----  
 0   id                float64
 1   asin              object 
 2   price             float64
 3   currency          object 
 4   time              object 
 5   crawlTime         object 
 6   condition         object 
 7   sellerName        object 
 8   sellerId          object 
 9   sellerbewertung   int64  
 10  seller_sterne     float64
 11  lieferdatum       object 
 12  lieferpreis       float64
 13  lieferung_durch   object 
 14  ranking           float64
 15  buyBoxWinner      bool   
 16  numberOfSellers   float64
 17  trigByReactive    object 
 18  Fulfillment_type  object 
 19  date_diff         float64
dtypes: bool(1), float64(7), int64(1), object(11)
memory usage: 961.7+ MB


In [2]:
## Print Basic Information about Dataframe.
#TimeFrame
df["time"] = pd.to_datetime(df["time"], format="%Y-%m-%d %H:%M:%S")
# Find the max and min values for the 'time' column
max_time = df['time'].max()
min_time = df['time'].min()

print("Min value for the 'time' column:", min_time)
print("Max value for the 'time' column:", max_time)


distinct_values_verkäufer = df['sellerName'].nunique()
print(f"Es gibt insgesamt {distinct_values_verkäufer} verschiedene Verkäufer")

Min value for the 'time' column: 2023-03-22 16:00:00
Max value for the 'time' column: 2023-04-23 23:00:00
Es gibt insgesamt 2377 verschiedene Verkäufer


## Generating List of asin Seller/Pairs and the Corresponding Price Changes/Observation Time

In [3]:
import pandas as pd
from datetime import timedelta

# Assuming your DataFrame is called df
df["time"] = pd.to_datetime(df["time"], format="%Y-%m-%d %H:%M:%S")

# First, sort the DataFrame by 'asin', 'sellerName', 'condition', 'time', and 'price'
df = df.sort_values(by=['asin', 'sellerName','condition', 'time','price'])

# Reset the index after sorting
df.reset_index(drop=True, inplace=True)

# Compute the price changes by checking if the price in the current row is different from the previous row,
# and if the sellerName is the same as in the previous row and if the condition is the same as in the previous row
# and if the time difference is exactly one hour and if there is no offer one hour earlier with the same condition and the same price from the same seller
df['price_change'] = (
    (df['price'] != df['price'].shift(1)) &
    (df['condition'] == df['condition'].shift(1)) &
    (df['sellerName'] == df['sellerName'].shift(1)) &
    (df['asin'] == df['asin'].shift(1)) &
    ((df['time'] - df['time'].shift(1)) == timedelta(hours=1)) &
    (~(
        ((df['condition'] == df['condition'].shift(1)) & (df['price'] == df['price'].shift(1)) & ((df['time'] - df['time'].shift(1)) == timedelta(hours=1)) & (df['sellerName'] == df['sellerName'].shift(1))) |
        ((df['condition'] == df['condition'].shift(2)) & (df['price'] == df['price'].shift(2)) & ((df['time'] - df['time'].shift(2)) == timedelta(hours=1)) & (df['sellerName'] == df['sellerName'].shift(2)))
    ))
).astype(int)

# Calculate the observation time for each seller-asin combination
observation_time = df.groupby(['asin', 'sellerName'])['time'].agg(['min', 'max']).reset_index()
observation_time['observation_time'] = observation_time['max'] - observation_time['min']

# Now, group by 'asin' and 'sellerName' and sum the 'price_change' column to get the number of price changes
price_changes = df.groupby(['asin', 'sellerName'])['price_change'].sum().reset_index()

# Merge the price_changes and observation_time DataFrames
result_df = pd.merge(price_changes, observation_time[['asin', 'sellerName', 'observation_time']], on=['asin', 'sellerName'])

# Convert the result to a list of tuples
result = list(result_df.itertuples(index=False, name=None))

# Print the result
for x in result:
    print(x)


('B00000JD6K', 'Kidia', 139, Timedelta('25 days 17:00:00'))
('B00000JD6K', 'STILE IMMAGINE DIGITAL HD', 0, Timedelta('29 days 02:00:00'))
('B00005KIRS', 'Amazon Warehouse ', 191, Timedelta('32 days 04:00:00'))
('B00005KIRS', 'Arlambo', 144, Timedelta('13 days 19:00:00'))
('B00005KIRS', 'Cernovia', 0, Timedelta('31 days 15:00:00'))
('B00005KIRS', 'ComSternde', 0, Timedelta('22 days 18:00:00'))
('B00005KIRS', 'Crystal4ever', 118, Timedelta('32 days 04:00:00'))
('B00005KIRS', 'DASTRO ® --- Einfach. Genial. Günstig.', 0, Timedelta('31 days 15:00:00'))
('B00005KIRS', 'Electronic XXL Sachsen', 0, Timedelta('30 days 05:00:00'))
('B00005KIRS', 'GREINER GROSSHANDEL', 168, Timedelta('32 days 04:00:00'))
('B00005KIRS', 'Kaleido.Shop GmbH', 180, Timedelta('31 days 19:00:00'))
('B00005KIRS', 'Technikprimus', 0, Timedelta('32 days 04:00:00'))
('B00005KIRS', 'amazon', 198, Timedelta('32 days 02:00:00'))
('B00005KIRS', 'e-joker', 117, Timedelta('32 days 04:00:00'))
('B00005KIRS', 'eibabo Deutschland',

## Actual Calculation if a seller is algorithmic

In [4]:
# Convert the result list to a DataFrame
result_df = pd.DataFrame(result, columns=['asin', 'sellerName', 'num_price_changes', 'observation_time'])

# Calculate the observation time in hours
result_df['observation_time'] = result_df['observation_time'].apply(lambda x: x.total_seconds() / 3600)

# Calculate the price changes per hour
result_df['price_changes_per_hour'] = result_df['num_price_changes'] / result_df['observation_time']

# Replace infinite values with NaN
result_df['price_changes_per_hour'] = result_df['price_changes_per_hour'].replace([np.inf, -np.inf], np.nan)

# Calculate the median of non-infinite and non-missing values
median_price_changes = result_df['price_changes_per_hour'].dropna().median()

# Replace missing and infinite values with the median
result_df['price_changes_per_hour'] = result_df['price_changes_per_hour'].fillna(median_price_changes)

# Calculate the mean and standard deviation
mean_price_changes = result_df['price_changes_per_hour'].mean()
std_price_changes = result_df['price_changes_per_hour'].std()

print("Mean price changes per hour:", mean_price_changes)
print("Standard deviation of price changes per hour:", std_price_changes)

# Set a threshold for classifying sellers as algorithmic
# For example, we can set the threshold as mean + 2 * std, which will classify approximately 2.5% of sellers as algorithmic
threshold = mean_price_changes + 1 * std_price_changes
print("Threshold:", threshold)

# Add a column to the result DataFrame indicating whether the seller is algorithmic
result_df['is_algorithmic'] = result_df['price_changes_per_hour'] > threshold

# Sort the algorithmic seller pairs by price changes per hour
algorithmic_seller_pairs = result_df[result_df['is_algorithmic']].sort_values(by='price_changes_per_hour', ascending=False)

# Get the names of algorithmic sellers
algorithmic_seller_names = algorithmic_seller_pairs['sellerName'].unique()

# Filter the result DataFrame to include only non-algorithmic sellers
non_algorithmic_seller_pairs = result_df[~result_df['sellerName'].isin(algorithmic_seller_names)]

# Combine the algorithmic and non-algorithmic seller pairs
final_result = pd.concat([algorithmic_seller_pairs, non_algorithmic_seller_pairs], ignore_index=True)


Mean price changes per hour: 0.06790070152020158
Standard deviation of price changes per hour: 0.11614839993296
Threshold: 0.18404910145316158


In [5]:
# Filter the DataFrame to include only the algorithmic sellers
algorithmic_sellers = final_result[final_result['is_algorithmic']]

# Get the unique seller names
unique_algorithmic_sellers = algorithmic_sellers['sellerName'].unique()

# Count the number of unique sellers by seller name
num_algorithmic_sellers = len(unique_algorithmic_sellers)

# Print the result
print("Number of unique algorithmic sellers:", num_algorithmic_sellers)
#print("Unique algorithmic seller names:")
#for seller_name in unique_algorithmic_sellers:
    #print(seller_name)


Number of unique algorithmic sellers: 628


In [6]:
# Filter the DataFrame to include only the non-algorithmic sellers
non_algorithmic_sellers = final_result[~final_result['is_algorithmic']]

# Get the unique seller names
unique_non_algorithmic_sellers = non_algorithmic_sellers['sellerName'].unique()

# Count the number of unique sellers by seller name
num_non_algorithmic_sellers = len(unique_non_algorithmic_sellers)
# Print the result
print("Number of unique non algorithmic sellers:", num_non_algorithmic_sellers,"of a total of ",distinct_values_verkäufer)
# Print the result
print("Number of unique algorithmic sellers:", num_algorithmic_sellers,"of a total of ",distinct_values_verkäufer)

Number of unique non algorithmic sellers: 1749 of a total of  2377
Number of unique algorithmic sellers: 628 of a total of  2377


# Calucalte Algorithmic Sellers by Total Price Changes over All Products

In [7]:
import pandas as pd

# Assuming your DataFrame is called df

# Convert the 'time' column to a 'datetime' type
df['time'] = pd.to_datetime(df['time'])

# First, sort the DataFrame by 'asin', 'sellerName', and 'time'
df = df.sort_values(by=['asin', 'sellerName', 'time'])

# Reset the index after sorting
df.reset_index(drop=True, inplace=True)

# Compute the price changes by checking if the price in the current row is different from the previous row, and if the sellerName is the same as in the previous row
df['price_change'] = (df['price'] != df['price'].shift(1)) & (df['sellerName'] == df['sellerName'].shift(1)) & (df['asin'] == df['asin'].shift(1))

# Now, group by 'sellerName' and sum the 'price_change' column to get the number of price changes
seller_price_changes = df.groupby('sellerName')['price_change'].sum().reset_index()

# Calculate the number of different products for each seller
unique_products = df.groupby('sellerName')['asin'].nunique().reset_index()

# Merge the seller_price_changes and unique_products DataFrames
seller_summary = pd.merge(seller_price_changes, unique_products, on='sellerName')

# Sort the results by the number of price changes in descending order
seller_summary = seller_summary.sort_values(by='price_change', ascending=False)

# Convert the result to a list of tuples
result_by_seller = [(row.sellerName, row.price_change, row.asin) for row in seller_summary.itertuples(index=False)]

# Print the result
for x in result_by_seller:
    print(x)


('Amazon Warehouse ', 297723, 448)
('amazon', 119631, 671)
('Happy Home Srl', 78707, 159)
('computeruniverse', 45905, 138)
('Solution 4YOU', 20197, 61)
('okluge', 19142, 81)
('GREINER GROSSHANDEL', 18251, 44)
('AS-Discount', 17910, 160)
('MusicScout', 16103, 34)
('e-joker', 13095, 155)
('Casa Haushalt', 12238, 42)
('TechPoint1111', 11732, 43)
('Stortle', 11315, 100)
('CW-SHOPPING', 10678, 42)
('Musikhaus Kirstein GmbH', 10311, 35)
('DaJa-Computer GbR', 9076, 159)
('cw-mobile', 8957, 68)
('ITCHECK24  --  alle Preise inkl. MwSt. - Widerrufsbelehrung + AGB unter Verkäuferhilfe', 8906, 51)
('Expert AGD', 8521, 53)
('To B To C', 8380, 17)
('nrsolutions', 8215, 84)
('SIA Solution', 7811, 41)
('nierlecom', 7548, 37)
('inandout', 7466, 100)
('GETIC', 6622, 8)
('GetMarket', 6412, 71)
('ProComponentes', 6393, 54)
('Happy Home Italia', 6280, 119)
('Computerhandlung', 5490, 56)
('myHobby24 | Ihr Freizeitspezialist für Deutschland', 5478, 68)
('price-guard', 5411, 164)
('Computerxtreme', 5341, 41)


## Calculation comes Now

In [8]:
import pandas as pd
import numpy as np

# Assuming result_by_seller is the list containing sellerName, price changes, and unique products

# Convert the list to a DataFrame
seller_df = pd.DataFrame(result_by_seller, columns=['sellerName', 'price_changes', 'unique_products'])

# Calculate the average number of price changes per product for each seller
seller_df['price_changes_per_product'] = seller_df['price_changes'] / seller_df['unique_products']

# Calculate the mean and standard deviation of the price changes per product
mean_changes_per_product = seller_df['price_changes_per_product'].mean()
std_changes_per_product = seller_df['price_changes_per_product'].std()

# Calculate the z-scores for each seller
seller_df['z_score'] = (seller_df['price_changes_per_product'] - mean_changes_per_product) / std_changes_per_product

# Define a manually adjustable threshold for the z-score
z_score_threshold = 1

# Classify sellers as algorithmic if their z-score is above the threshold
seller_df['is_algorithmic'] = seller_df['z_score'] > z_score_threshold

# Print the z-score threshold
print('Z-score Threshold:', z_score_threshold)

# Print the DataFrame with the algorithmic sellers
algorithmic_sellers_grouped = seller_df[seller_df['is_algorithmic']]


Z-score Threshold: 1


In [9]:
import pandas as pd
import numpy as np

# Assuming result_by_seller is the list containing sellerName, price changes, and unique products

# Convert the list to a DataFrame
seller_df = pd.DataFrame(result_by_seller, columns=['sellerName', 'price_changes', 'unique_products'])

# Calculate the percentiles for price_changes and unique_products
seller_df['price_changes_percentile'] = seller_df['price_changes'].rank(pct=True)
seller_df['unique_products_percentile'] = seller_df['unique_products'].rank(pct=True)

# Calculate the z-scores for the percentiles
seller_df['z_score_price_changes'] = (seller_df['price_changes_percentile'] - seller_df['price_changes_percentile'].mean()) / seller_df['price_changes_percentile'].std()
seller_df['z_score_unique_products'] = (seller_df['unique_products_percentile'] - seller_df['unique_products_percentile'].mean()) / seller_df['unique_products_percentile'].std()

# Define weights for the z-scores of price_changes and unique_products
weight_price_changes = 1.5
weight_unique_products = 0.5

# Combine the z-scores to create a combined score
seller_df['combined_z_score'] = (weight_price_changes * seller_df['z_score_price_changes']) + (weight_unique_products * seller_df['z_score_unique_products'])

# Define a manually adjustable threshold for the combined z-score
combined_z_score_threshold = 2.5  # Increase the threshold to reduce the number of algorithmic sellers

# Classify sellers as algorithmic if their combined z-score is above the threshold
seller_df['is_algorithmic'] = seller_df['combined_z_score'] > combined_z_score_threshold

# Print the combined z-score threshold
print('Combined Z-score Threshold:', combined_z_score_threshold)

# Print the DataFrame with the algorithmic sellers
algorithmic_sellers_grouped = seller_df[seller_df['is_algorithmic']]


Combined Z-score Threshold: 2.5


## Chen Method with simple numberic Treshhold

In [10]:
algo_seller_chen = []

for tup in result:
    number_of_price_changes = tup[2]
    sellerName = tup[1]
    if number_of_price_changes > 20:
        #print(tup)
        algo_seller_chen.append(sellerName)

# Remove duplicates by converting the list to a set and then back to a list
algo_seller_chen = list(set(algo_seller_chen))
# Transform the list into a DataFrame
df_algo_seller_chen = pd.DataFrame(algo_seller_chen, columns=['sellerName'])

print("Length of algo_seller_chen list (without duplicates):", len(df_algo_seller_chen))

Length of algo_seller_chen list (without duplicates): 862


## Comparing how many Algorithmic Sellers both Ways have in Common

In [11]:
import pandas as pd

# Remove duplicate sellerName entries from algorithmic_sellers
unique_algorithmic_sellers = algorithmic_sellers.drop_duplicates(subset='sellerName')

# Calculate the length of the unique_algorithmic_sellers DataFrame and the algorithmic_sellers_grouped DataFrame
len_unique_algorithmic_sellers = len(unique_algorithmic_sellers)
len_algorithmic_sellers_grouped = len(algorithmic_sellers_grouped)
len_algorithmic_seller_chen = len(df_algo_seller_chen)

# Merge the DataFrames using an inner join, keeping only the common rows
common_sellers_df = unique_algorithmic_sellers.merge(algorithmic_sellers_grouped, on='sellerName', how='inner')
common_sellers_chen_unique = unique_algorithmic_sellers.merge(df_algo_seller_chen, on='sellerName', how='inner')

# Calculate the percentage of common sellers
percentage_common_sellers = (len(common_sellers_df) / min(len_unique_algorithmic_sellers, len_algorithmic_sellers_grouped)) * 100
percentage_common_seller_chen_unique = (len(common_sellers_chen_unique) / min(len_unique_algorithmic_sellers, len_algorithmic_seller_chen)) * 100
percentage_on_all_sellers = (len(common_sellers_chen_unique) / distinct_values_verkäufer) *100
print(f"Total Number of Sellers:", distinct_values_verkäufer)


# Print the lengths of the DataFrames and the percentage of common sellers
print("1: Length of unique_algorithmic_sellers:", len_unique_algorithmic_sellers)
print("2: Length of algorithmic_sellers_grouped:", len_algorithmic_sellers_grouped)
print("3: Length of algortihmic_seller_chen:", len(algo_seller_chen))
print("Length of Sellers in Common between 1 and 2:", len(common_sellers_df))
print("Percentage of common sellers between 1 and 2:", percentage_common_sellers)
print("Percentage of common sellers 1 and 3:", percentage_common_seller_chen_unique)
print("Percentage on all Sellers: ", percentage_on_all_sellers )


# Print the common SellerNames as a list
common_sellers_list = common_sellers_chen_unique['sellerName'].tolist()
#print("\nCommon SellerNames:")
#print(common_sellers_list)
print(len(common_sellers_list))
# Convert the list to a pandas DataFrame
df_algorithmic_seller_names = pd.DataFrame(common_sellers_list, columns=['AlgorithmicSellerNames'])
df_algorithmic_seller_names.to_csv('../Grunddatein/Zwischendatein/PA-Adopters.csv', index=False)

Total Number of Sellers: 2377
1: Length of unique_algorithmic_sellers: 628
2: Length of algorithmic_sellers_grouped: 261
3: Length of algortihmic_seller_chen: 862
Length of Sellers in Common between 1 and 2: 196
Percentage of common sellers between 1 and 2: 75.09578544061303
Percentage of common sellers 1 and 3: 97.29299363057325
Percentage on all Sellers:  25.704669751787968
611


## Sellers only present in the first version or the second version

In [12]:
import pandas as pd

# Merge the DataFrames using an outer join, keeping the origin information in the _merge column
merged_df = algorithmic_sellers.merge(algorithmic_sellers_grouped, on='sellerName', how='outer', indicator=True)

# Filter the merged DataFrame to find sellers unique to each DataFrame
unique_sellers_algorithmic_sellers = merged_df[merged_df['_merge'] == 'left_only']
unique_sellers_algorithmic_sellers_grouped = merged_df[merged_df['_merge'] == 'right_only']

# Convert the filtered Series to lists without index
unique_sellers_list_algorithmic_sellers = unique_sellers_algorithmic_sellers['sellerName'].tolist()
unique_sellers_list_algorithmic_sellers_grouped = unique_sellers_algorithmic_sellers_grouped['sellerName'].tolist()

# Print the sellers unique to each DataFrame as lists without index
print("Sellers only present in algorithmic_sellers:")
#print(unique_sellers_list_algorithmic_sellers)
print(len(unique_sellers_list_algorithmic_sellers))

print("\nSellers only present in algorithmic_sellers_grouped:")
#print(unique_sellers_list_algorithmic_sellers_grouped)
print(len(unique_sellers_algorithmic_sellers_grouped))

Sellers only present in algorithmic_sellers:
556

Sellers only present in algorithmic_sellers_grouped:
65
